## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Punta Arenas,CL,-53.15,-70.92,48.20,61,75,5.82,broken clouds
1,1,Vaini,TO,-21.20,-175.20,75.20,73,75,9.17,broken clouds
2,2,Bluff,NZ,-46.60,168.33,60.01,56,95,7.00,overcast clouds
3,3,Takoradi,GH,4.88,-1.76,76.69,92,65,7.05,broken clouds
4,4,Sorland,NO,67.67,12.69,37.04,78,92,11.54,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
1,1,Vaini,TO,-21.20,-175.20,75.20,73,75,9.17,broken clouds
3,3,Takoradi,GH,4.88,-1.76,76.69,92,65,7.05,broken clouds
7,7,Bengkulu,ID,-3.80,102.27,81.01,72,100,5.86,overcast clouds
8,8,Sur,OM,22.57,59.53,78.40,59,2,8.81,clear sky
11,11,Bathsheba,BB,13.22,-59.52,82.40,83,75,17.22,broken clouds
13,13,Butaritari,KI,3.07,172.79,82.00,71,13,8.81,few clouds
14,14,Mahebourg,MU,-20.41,57.70,75.20,73,40,14.99,shower rain
25,25,Isangel,VU,-19.55,169.27,81.23,69,4,9.33,light rain
26,26,Saint-Philippe,RE,-21.36,55.77,75.20,57,59,4.70,broken clouds
36,36,Kapaa,US,22.08,-159.32,82.40,74,75,9.17,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current description    211
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()



In [8]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
3,Takoradi,GH,76.69,broken clouds,4.88,-1.76,
7,Bengkulu,ID,81.01,overcast clouds,-3.80,102.27,
8,Sur,OM,78.40,clear sky,22.57,59.53,
11,Bathsheba,BB,82.40,broken clouds,13.22,-59.52,
13,Butaritari,KI,82.00,few clouds,3.07,172.79,
14,Mahebourg,MU,75.20,shower rain,-20.41,57.70,
25,Isangel,VU,81.23,light rain,-19.55,169.27,
26,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,
36,Kapaa,US,82.40,broken clouds,22.08,-159.32,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
   
        

In [10]:
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(20)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
3,Takoradi,GH,76.69,broken clouds,4.88,-1.76,Raybow International Hotel
7,Bengkulu,ID,81.01,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
8,Sur,OM,78.40,clear sky,22.57,59.53,Sur Plaza Hotel
11,Bathsheba,BB,82.40,broken clouds,13.22,-59.52,Atlantis Hotel
13,Butaritari,KI,82.00,few clouds,3.07,172.79,Isles Sunset Lodge
14,Mahebourg,MU,75.20,shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
25,Isangel,VU,81.23,light rain,-19.55,169.27,Volcano Sea View Tree House
26,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
36,Kapaa,US,82.40,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)



hotel_df.dropna(subset=['Hotel Name'], inplace=True)


In [13]:
hotel_df.head(20)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
3,Takoradi,GH,76.69,broken clouds,4.88,-1.76,Raybow International Hotel
7,Bengkulu,ID,81.01,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
8,Sur,OM,78.40,clear sky,22.57,59.53,Sur Plaza Hotel
11,Bathsheba,BB,82.40,broken clouds,13.22,-59.52,Atlantis Hotel
13,Butaritari,KI,82.00,few clouds,3.07,172.79,Isles Sunset Lodge
14,Mahebourg,MU,75.20,shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
25,Isangel,VU,81.23,light rain,-19.55,169.27,Volcano Sea View Tree House
26,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
36,Kapaa,US,82.40,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


In [20]:
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""




In [23]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))